In [1]:
import torch
import gc
import logging

# del session.logger
# del session.epoch_logger
# del session
# gc.collect()
# torch.cuda.empty_cache()

import multiprocessing
from train.training_session_v1 import TrainingSessionV1
from config import TrainingConfigV1
from config import SimpleConvConfig

data_partition = {
    "gwilliams2023": {
        "testing_subjects": [19, 20, 21],
        "testing_tasks": [0],
    },
    # "armeini2022": {
    #     "testing_subjects": [],
    #     "testing_tasks": [8, 9],
    # },
}

model_config = SimpleConvConfig(
    # Str to list of possible conditions
    mel_normalization=False,
    conditions={
        "study": [],
        "subject": [],
    },
    # Channels
    in_channels=208,
    out_channels=80,
    hidden_dim=384,
    dropout=0.2,
    initial_batch_norm=True,
    # Sensor layout settings
    layout_dim=2,
    layout_proj=True,
    layout_scaling="minmax",
    # Merger with spatial attn
    merger=False,
    merger_emb_type=None,
    merger_emb_dim=0,
    merger_channels=0,
    merger_dropout=0.0,  # Float
    merger_conditional=None,
    # Inital
    initial_linear=384,
    initial_depth=1,
    # Conditional layers
    conditional_layers=False,
    conditional_layers_dim=None,  # input or hidden_dim
    # Conv layer overall structure
    depth=6,
    kernel_size=3,
    growth=1.0,
    dilation_growth=2,
    dilation_period=5,
    glu=1,
    conv_dropout=0.2,
    dropout_input=0.2,
    batch_norm=True,
    half=True,
    cnn_pos_encoding=False,
    # Quantizer
    quantizer=False,
    num_codebooks=0,
    codebook_size=0,
    quantizer_commitment=0,
    quantizer_temp_init=0,
    quantizer_temp_min=0,
    quantizer_temp_decay=0,
    # Transformers Encoders
    transformer_input="continuous",
    transformer_encoder_emb="sinusoidal",
    transformer_encoder_layers=1,
    transformer_encoder_heads=4,
    # Conformer encoder variant
    rnn_type="transformer",
    depthwise_conv_kernel_size=0,
    use_group_norm=False,
    convolution_first=False,
    # Transformer Decoders
    transformer_decoder_emb=None,
    transformer_decoder_layers=0,
    transformer_decoder_heads=0,
    transformer_decoder_dim=0,
)

config = TrainingConfigV1(
    brain_encoder_config=model_config,
    data_partition=data_partition,
    # Ada lora settings
    # Around 100k total batches an epoch for gwilliams
    adalora_init_r=12,
    adalora_target_r=4,
    adalora_tinit=(400 * 3),  # 5% total steps
    adalora_tfinal=(400 * 8),  # 50-80% total steps
    adalora_deltaT=(400 * 1),  # 1-5% total steps
    adalora_lora_alpha=32,
    adalora_lora_dropout=0.1,
    adalora_total_step=(400 * 30),
    # Pre-processing parameters
    # Brain
    new_freq=200,
    frequency_bands={"all": (0.5, 80)},
    max_random_shift=1.0,
    window_size=4,
    window_stride=1,
    brain_clipping=None,
    baseline_window=0.5,
    notch_filter=True,
    scaling="both",
    delay=0.15,
    # Audio
    audio_model="openai/whisper-tiny.en",
    # Hyperparameters
    learning_rate=1e-4,
    weight_decay=1e-4,
    epochs=40,
    batch_size=128,
    random_test_size=10,
    seed=42,
    mel_alignment_objectives={
        "clip_loss": 0.6,
        "mse_loss": 0.4,
        "commitment_loss": 0.0,
        "cosine_similarity": 0.0,
    },
    latent_alignment_objectives={
        "cosine_similarity": 4.0,
        "mse_loss": 4.0,
        "clip_loss": 0.2,
    },
    latent_alignment_layers=[-1],
)

config.brain_encoder_config.mel_normalization = False
config.learning_rate = 1e-4
config.batch_size = 256

session = TrainingSessionV1(
    config=config,
    studies={study: "audio" for study in data_partition.keys()},
    data_path="/home/ubuntu/storage-texas/data",
    save_path="saves/phase2/architecture/task/transformers/1_transformer_layer",
    clear_cache=False,
    cache_name="cache",
    download_studies=False,
)

try:
    session.train(
        device="cuda",
        buffer_size=30,
        num_workers=(multiprocessing.cpu_count() - 2),
        max_cache_size=400,
        current_epoch=0,
    )
except KeyboardInterrupt as e:
    print("Exited")

# try:
#     session.pre_process_all_recordings(
#         buffer_size=30, num_workers=multiprocessing.cpu_count() - 2, max_cache_size=400
#     )
# except KeyboardInterrupt as e:
#     print("Exited")

Loading Gwilliams2023 with batch type audio
Data partitioned on studies ['gwilliams2023'].
Train: 135, Unseen Task: 12, Unseen Subject: 45, Unseen Both: 4.

RNNEncoder initialized as transformer with 1 layers, 384 d_model, 4 nhead
	Embedding: sinusoidal, params: 1774464
SimpleConv initialized with 11070448 parameters, cond: ['study', 'subject']
Merger False, merger channels 0
ConvBlocks: 6, hidden_dim: 384, params 8858112
Found 24 target modules for AdaLora: ['k_proj', 'q_proj', 'v_proj', 'out_proj', 'fc1', 'fc2']
openai/whisper-tiny.en loaded with 8540472 frozen params (4 layers and 384 dim).
AdaLora has 332064 trainable params.


2025-02-11 00:56:17,416	INFO worker.py:1841 -- Started a local Ray instance.
Training Epoch 1: 100%|██████████| 135/135 [07:50<00:00,  3.49s/it]


Testing completed in 0.66m.
Epoch 1 completed in 8.53m. 0.06m per recording.
New highest average test accuracy: 0.0051, lowest FinLayer total loss: 4.9590 at epoch 1.


Training Epoch 2: 100%|██████████| 135/135 [07:41<00:00,  3.42s/it]


Testing completed in 0.66m.
Epoch 2 completed in 8.35m. 0.06m per recording.
New highest average test accuracy: 0.0184, lowest FinLayer total loss: 4.8309 at epoch 2.


...

Testing completed in 0.67m.
Epoch 38 completed in 8.36m. 0.06m per recording.


Training Epoch 39: 100%|██████████| 135/135 [07:43<00:00,  3.43s/it]


Testing completed in 0.67m.
Epoch 39 completed in 8.39m. 0.06m per recording.


Training Epoch 40: 100%|██████████| 135/135 [07:42<00:00,  3.43s/it]


Testing completed in 0.66m.
Epoch 40 completed in 8.37m. 0.06m per recording.
New highest average test accuracy: 0.0039, lowest FinLayer total loss: 4.5568 at epoch 40.
Training completed.
unseen_subject: Acc: 0.0045, Top 5: 0.0290, Top 10: 0.0515
Loss: 10.2799, Mel Loss: 5.7401, Clip Loss: 9.4415, MSE Loss: 0.1880, CosSim Loss: 0.0000, Commit Loss: 0.0000
FinLayer Clip Loss: 6.9879, FinLayer MSE Loss: 0.5697
FinLayer CosSim Loss: 0.2158, FinLayer Total Loss: 4.5398
unseen_task: Acc: 0.0047, Top 5: 0.0230, Top 10: 0.0454
Loss: 10.3823, Mel Loss: 5.7923, Clip Loss: 9.5191, MSE Loss: 0.2020, CosSim Loss: 0.0000, Commit Loss: 0.0000
FinLayer Clip Loss: 7.0563, FinLayer MSE Loss: 0.5770
FinLayer CosSim Loss: 0.2177, FinLayer Total Loss: 4.5900
unseen_both: Acc: 0.0025, Top 5: 0.0325, Top 10: 0.0663
Loss: 10.2820, Mel Loss: 5.7414, Clip Loss: 9.4437, MSE Loss: 0.1879, CosSim Loss: 0.0000, Commit Loss: 0.0000
FinLayer Clip Loss: 6.9903, FinLayer MSE Loss: 0.5698
FinLayer CosSim Loss: 0.2158,

In [2]:
import torch
import gc
import logging

del session.logger
del session.epoch_logger
del session
gc.collect()
torch.cuda.empty_cache()

import multiprocessing
from train.training_session_v1 import TrainingSessionV1
from config import TrainingConfigV1
from config import SimpleConvConfig

data_partition = {
    "gwilliams2023": {
        "testing_subjects": [19, 20, 21],
        "testing_tasks": [0],
    },
    # "armeini2022": {
    #     "testing_subjects": [],
    #     "testing_tasks": [8, 9],
    # },
}

model_config = SimpleConvConfig(
    # Str to list of possible conditions
    mel_normalization=False,
    conditions={
        "study": [],
        "subject": [],
    },
    # Channels
    in_channels=208,
    out_channels=80,
    hidden_dim=384,
    dropout=0.2,
    initial_batch_norm=True,
    # Sensor layout settings
    layout_dim=2,
    layout_proj=True,
    layout_scaling="minmax",
    # Merger with spatial attn
    merger=False,
    merger_emb_type=None,
    merger_emb_dim=0,
    merger_channels=0,
    merger_dropout=0.0,  # Float
    merger_conditional=None,
    # Inital
    initial_linear=384,
    initial_depth=1,
    # Conditional layers
    conditional_layers=False,
    conditional_layers_dim=None,  # input or hidden_dim
    # Conv layer overall structure
    depth=6,
    kernel_size=3,
    growth=1.0,
    dilation_growth=2,
    dilation_period=5,
    glu=1,
    conv_dropout=0.2,
    dropout_input=0.2,
    batch_norm=True,
    half=True,
    cnn_pos_encoding=False,
    # Quantizer
    quantizer=False,
    num_codebooks=0,
    codebook_size=0,
    quantizer_commitment=0,
    quantizer_temp_init=0,
    quantizer_temp_min=0,
    quantizer_temp_decay=0,
    # Transformers Encoders
    transformer_input="continuous",
    transformer_encoder_emb="sinusoidal",
    transformer_encoder_layers=2,
    transformer_encoder_heads=4,
    # Conformer encoder variant
    rnn_type="transformer",
    depthwise_conv_kernel_size=0,
    use_group_norm=False,
    convolution_first=False,
    # Transformer Decoders
    transformer_decoder_emb=None,
    transformer_decoder_layers=0,
    transformer_decoder_heads=0,
    transformer_decoder_dim=0,
)

config = TrainingConfigV1(
    brain_encoder_config=model_config,
    data_partition=data_partition,
    # Ada lora settings
    # Around 100k total batches an epoch for gwilliams
    adalora_init_r=12,
    adalora_target_r=4,
    adalora_tinit=(400 * 3),  # 5% total steps
    adalora_tfinal=(400 * 8),  # 50-80% total steps
    adalora_deltaT=(400 * 1),  # 1-5% total steps
    adalora_lora_alpha=32,
    adalora_lora_dropout=0.1,
    adalora_total_step=(400 * 30),
    # Pre-processing parameters
    # Brain
    new_freq=200,
    frequency_bands={"all": (0.5, 80)},
    max_random_shift=1.0,
    window_size=4,
    window_stride=1,
    brain_clipping=None,
    baseline_window=0.5,
    notch_filter=True,
    scaling="both",
    delay=0.15,
    # Audio
    audio_model="openai/whisper-tiny.en",
    # Hyperparameters
    learning_rate=1e-4,
    weight_decay=1e-4,
    epochs=40,
    batch_size=128,
    random_test_size=10,
    seed=42,
    mel_alignment_objectives={
        "clip_loss": 0.6,
        "mse_loss": 0.4,
        "commitment_loss": 0.0,
        "cosine_similarity": 0.0,
    },
    latent_alignment_objectives={
        "cosine_similarity": 4.0,
        "mse_loss": 4.0,
        "clip_loss": 0.2,
    },
    latent_alignment_layers=[-1],
)

config.brain_encoder_config.mel_normalization = False
config.learning_rate = 1e-4
config.batch_size = 256

session = TrainingSessionV1(
    config=config,
    studies={study: "audio" for study in data_partition.keys()},
    data_path="/home/ubuntu/storage-texas/data",
    save_path="saves/phase2/architecture/task/transformers/2_transformer_layer",
    clear_cache=False,
    cache_name="cache",
    download_studies=False,
)

try:
    session.train(
        device="cuda",
        buffer_size=30,
        num_workers=(multiprocessing.cpu_count() - 2),
        max_cache_size=400,
        current_epoch=0,
    )
except KeyboardInterrupt as e:
    print("Exited")

# try:
#     session.pre_process_all_recordings(
#         buffer_size=30, num_workers=multiprocessing.cpu_count() - 2, max_cache_size=400
#     )
# except KeyboardInterrupt as e:
#     print("Exited")

Loading Gwilliams2023 with batch type audio
Data partitioned on studies ['gwilliams2023'].
Train: 135, Unseen Task: 12, Unseen Subject: 45, Unseen Both: 4.

RNNEncoder initialized as transformer with 2 layers, 384 d_model, 4 nhead
	Embedding: sinusoidal, params: 3548928
SimpleConv initialized with 12844912 parameters, cond: ['study', 'subject']
Merger False, merger channels 0
ConvBlocks: 6, hidden_dim: 384, params 8858112
Found 24 target modules for AdaLora: ['k_proj', 'q_proj', 'v_proj', 'out_proj', 'fc1', 'fc2']
openai/whisper-tiny.en loaded with 8540472 frozen params (4 layers and 384 dim).
AdaLora has 332064 trainable params.


Training Epoch 1: 100%|██████████| 135/135 [07:50<00:00,  3.48s/it]


Testing completed in 0.67m.
Epoch 1 completed in 8.51m. 0.06m per recording.
New highest average test accuracy: 0.0048, lowest FinLayer total loss: 4.9691 at epoch 1.


Training Epoch 2: 100%|██████████| 135/135 [07:51<00:00,  3.49s/it]


Testing completed in 0.67m.
Epoch 2 completed in 8.52m. 0.06m per recording.
New highest average test accuracy: 0.0052, lowest FinLayer total loss: 4.8789 at epoch 2.


...

Testing completed in 0.66m.
Epoch 17 completed in 8.47m. 0.06m per recording.


Training Epoch 18: 100%|██████████| 135/135 [07:49<00:00,  3.48s/it]


Testing completed in 0.66m.
Epoch 18 completed in 8.49m. 0.06m per recording.


Training Epoch 19: 100%|██████████| 135/135 [07:49<00:00,  3.48s/it]


Testing completed in 0.67m.
Epoch 19 completed in 8.49m. 0.06m per recording.
Early stopping at epoch 19. Highest top 10 accuracy at epoch 8.
Training completed.
unseen_subject: Acc: 0.0805, Top 5: 0.2335, Top 10: 0.3375
Loss: 9.9860, Mel Loss: 5.6942, Clip Loss: 9.3654, MSE Loss: 0.1874, CosSim Loss: 0.0000, Commit Loss: 0.0000
FinLayer Clip Loss: 6.0708, FinLayer MSE Loss: 0.5595
FinLayer CosSim Loss: 0.2100, FinLayer Total Loss: 4.2918
unseen_task: Acc: 0.0776, Top 5: 0.1971, Top 10: 0.2780
Loss: 10.0551, Mel Loss: 5.7401, Clip Loss: 9.4341, MSE Loss: 0.1992, CosSim Loss: 0.0000, Commit Loss: 0.0000
FinLayer Clip Loss: 6.0322, FinLayer MSE Loss: 0.5657
FinLayer CosSim Loss: 0.2114, FinLayer Total Loss: 4.3150
unseen_both: Acc: 0.0625, Top 5: 0.1900, Top 10: 0.2800
Loss: 10.0175, Mel Loss: 5.7080, Clip Loss: 9.3870, MSE Loss: 0.1896, CosSim Loss: 0.0000, Commit Loss: 0.0000
FinLayer Clip Loss: 6.1025, FinLayer MSE Loss: 0.5615
FinLayer CosSim Loss: 0.2107, FinLayer Total Loss: 4.3094

In [3]:
import torch
import gc
import logging

del session.logger
del session.epoch_logger
del session
gc.collect()
torch.cuda.empty_cache()

import multiprocessing
from train.training_session_v1 import TrainingSessionV1
from config import TrainingConfigV1
from config import SimpleConvConfig

data_partition = {
    "gwilliams2023": {
        "testing_subjects": [19, 20, 21],
        "testing_tasks": [0],
    },
    # "armeini2022": {
    #     "testing_subjects": [],
    #     "testing_tasks": [8, 9],
    # },
}

model_config = SimpleConvConfig(
    # Str to list of possible conditions
    mel_normalization=False,
    conditions={
        "study": [],
        "subject": [],
    },
    # Channels
    in_channels=208,
    out_channels=80,
    hidden_dim=384,
    dropout=0.2,
    initial_batch_norm=True,
    # Sensor layout settings
    layout_dim=2,
    layout_proj=True,
    layout_scaling="minmax",
    # Merger with spatial attn
    merger=False,
    merger_emb_type=None,
    merger_emb_dim=0,
    merger_channels=0,
    merger_dropout=0.0,  # Float
    merger_conditional=None,
    # Inital
    initial_linear=384,
    initial_depth=1,
    # Conditional layers
    conditional_layers=False,
    conditional_layers_dim=None,  # input or hidden_dim
    # Conv layer overall structure
    depth=6,
    kernel_size=3,
    growth=1.0,
    dilation_growth=2,
    dilation_period=5,
    glu=1,
    conv_dropout=0.2,
    dropout_input=0.2,
    batch_norm=True,
    half=True,
    cnn_pos_encoding=False,
    # Quantizer
    quantizer=False,
    num_codebooks=0,
    codebook_size=0,
    quantizer_commitment=0,
    quantizer_temp_init=0,
    quantizer_temp_min=0,
    quantizer_temp_decay=0,
    # Transformers Encoders
    transformer_input="continuous",
    transformer_encoder_emb="sinusoidal",
    transformer_encoder_layers=3,
    transformer_encoder_heads=4,
    # Conformer encoder variant
    rnn_type="transformer",
    depthwise_conv_kernel_size=0,
    use_group_norm=False,
    convolution_first=False,
    # Transformer Decoders
    transformer_decoder_emb=None,
    transformer_decoder_layers=0,
    transformer_decoder_heads=0,
    transformer_decoder_dim=0,
)

config = TrainingConfigV1(
    brain_encoder_config=model_config,
    data_partition=data_partition,
    # Ada lora settings
    # Around 100k total batches an epoch for gwilliams
    adalora_init_r=12,
    adalora_target_r=4,
    adalora_tinit=(400 * 3),  # 5% total steps
    adalora_tfinal=(400 * 8),  # 50-80% total steps
    adalora_deltaT=(400 * 1),  # 1-5% total steps
    adalora_lora_alpha=32,
    adalora_lora_dropout=0.1,
    adalora_total_step=(400 * 30),
    # Pre-processing parameters
    # Brain
    new_freq=200,
    frequency_bands={"all": (0.5, 80)},
    max_random_shift=1.0,
    window_size=4,
    window_stride=1,
    brain_clipping=None,
    baseline_window=0.5,
    notch_filter=True,
    scaling="both",
    delay=0.15,
    # Audio
    audio_model="openai/whisper-tiny.en",
    # Hyperparameters
    learning_rate=1e-4,
    weight_decay=1e-4,
    epochs=40,
    batch_size=128,
    random_test_size=10,
    seed=42,
    mel_alignment_objectives={
        "clip_loss": 0.6,
        "mse_loss": 0.4,
        "commitment_loss": 0.0,
        "cosine_similarity": 0.0,
    },
    latent_alignment_objectives={
        "cosine_similarity": 4.0,
        "mse_loss": 4.0,
        "clip_loss": 0.2,
    },
    latent_alignment_layers=[-1],
)

config.brain_encoder_config.mel_normalization = False
config.learning_rate = 1e-4
config.batch_size = 256

session = TrainingSessionV1(
    config=config,
    studies={study: "audio" for study in data_partition.keys()},
    data_path="/home/ubuntu/storage-texas/data",
    save_path="saves/phase2/architecture/task/transformers/3_transformer_layer",
    clear_cache=False,
    cache_name="cache",
    download_studies=False,
)

try:
    session.train(
        device="cuda",
        buffer_size=30,
        num_workers=(multiprocessing.cpu_count() - 2),
        max_cache_size=400,
        current_epoch=0,
    )
except KeyboardInterrupt as e:
    print("Exited")

# try:
#     session.pre_process_all_recordings(
#         buffer_size=30, num_workers=multiprocessing.cpu_count() - 2, max_cache_size=400
#     )
# except KeyboardInterrupt as e:
#     print("Exited")

Loading Gwilliams2023 with batch type audio
Data partitioned on studies ['gwilliams2023'].
Train: 135, Unseen Task: 12, Unseen Subject: 45, Unseen Both: 4.

RNNEncoder initialized as transformer with 3 layers, 384 d_model, 4 nhead
	Embedding: sinusoidal, params: 5323392
SimpleConv initialized with 14619376 parameters, cond: ['study', 'subject']
Merger False, merger channels 0
ConvBlocks: 6, hidden_dim: 384, params 8858112
Found 24 target modules for AdaLora: ['k_proj', 'q_proj', 'v_proj', 'out_proj', 'fc1', 'fc2']
openai/whisper-tiny.en loaded with 8540472 frozen params (4 layers and 384 dim).
AdaLora has 332064 trainable params.


Training Epoch 1: 100%|██████████| 135/135 [07:55<00:00,  3.52s/it]


Testing completed in 0.66m.
Epoch 1 completed in 8.58m. 0.06m per recording.
New highest average test accuracy: 0.0048, lowest FinLayer total loss: 4.9703 at epoch 1.


Training Epoch 2: 100%|██████████| 135/135 [07:54<00:00,  3.52s/it]


Testing completed in 0.67m.
Epoch 2 completed in 8.58m. 0.06m per recording.
New highest average test accuracy: 0.0048, lowest FinLayer total loss: 4.8615 at epoch 2.


...

Testing completed in 0.66m.
Epoch 17 completed in 8.58m. 0.06m per recording.


Training Epoch 18: 100%|██████████| 135/135 [07:53<00:00,  3.51s/it]


Testing completed in 0.67m.
Epoch 18 completed in 8.56m. 0.06m per recording.


Training Epoch 19: 100%|██████████| 135/135 [07:53<00:00,  3.51s/it]


Testing completed in 0.66m.
Epoch 19 completed in 8.55m. 0.06m per recording.
Early stopping at epoch 19. Highest top 10 accuracy at epoch 8.
Training completed.
unseen_subject: Acc: 0.0520, Top 5: 0.1480, Top 10: 0.2115
Loss: 10.1461, Mel Loss: 5.7276, Clip Loss: 9.4111, MSE Loss: 0.2024, CosSim Loss: 0.0000, Commit Loss: 0.0000
FinLayer Clip Loss: 6.4135, FinLayer MSE Loss: 0.5693
FinLayer CosSim Loss: 0.2147, FinLayer Total Loss: 4.4185
unseen_task: Acc: 0.0344, Top 5: 0.1101, Top 10: 0.1826
Loss: 10.2269, Mel Loss: 5.7713, Clip Loss: 9.4761, MSE Loss: 0.2143, CosSim Loss: 0.0000, Commit Loss: 0.0000
FinLayer Clip Loss: 6.4178, FinLayer MSE Loss: 0.5764
FinLayer CosSim Loss: 0.2166, FinLayer Total Loss: 4.4556
unseen_both: Acc: 0.0388, Top 5: 0.1200, Top 10: 0.1825
Loss: 10.1480, Mel Loss: 5.7286, Clip Loss: 9.4122, MSE Loss: 0.2032, CosSim Loss: 0.0000, Commit Loss: 0.0000
FinLayer Clip Loss: 6.4104, FinLayer MSE Loss: 0.5696
FinLayer CosSim Loss: 0.2148, FinLayer Total Loss: 4.419

In [ ]:
import torch
import gc
import logging

del session.logger
del session.epoch_logger
del session
gc.collect()
torch.cuda.empty_cache()

import multiprocessing
from train.training_session_v1 import TrainingSessionV1
from config import TrainingConfigV1
from config import SimpleConvConfig

data_partition = {
    "gwilliams2023": {
        "testing_subjects": [19, 20, 21],
        "testing_tasks": [0],
    },
    # "armeini2022": {
    #     "testing_subjects": [],
    #     "testing_tasks": [8, 9],
    # },
}

model_config = SimpleConvConfig(
    # Str to list of possible conditions
    mel_normalization=False,
    conditions={
        "study": [],
        "subject": [],
    },
    # Channels
    in_channels=208,
    out_channels=80,
    hidden_dim=384,
    dropout=0.2,
    initial_batch_norm=True,
    # Sensor layout settings
    layout_dim=2,
    layout_proj=True,
    layout_scaling="minmax",
    # Merger with spatial attn
    merger=False,
    merger_emb_type=None,
    merger_emb_dim=0,
    merger_channels=0,
    merger_dropout=0.0,  # Float
    merger_conditional=None,
    # Inital
    initial_linear=384,
    initial_depth=1,
    # Conditional layers
    conditional_layers=False,
    conditional_layers_dim=None,  # input or hidden_dim
    # Conv layer overall structure
    depth=6,
    kernel_size=3,
    growth=1.0,
    dilation_growth=2,
    dilation_period=5,
    glu=1,
    conv_dropout=0.2,
    dropout_input=0.2,
    batch_norm=True,
    half=True,
    cnn_pos_encoding=False,
    # Quantizer
    quantizer=False,
    num_codebooks=0,
    codebook_size=0,
    quantizer_commitment=0,
    quantizer_temp_init=0,
    quantizer_temp_min=0,
    quantizer_temp_decay=0,
    # Transformers Encoders
    transformer_input="continuous",
    transformer_encoder_emb="sinusoidal",
    transformer_encoder_layers=4,
    transformer_encoder_heads=4,
    # Conformer encoder variant
    rnn_type="transformer",
    depthwise_conv_kernel_size=0,
    use_group_norm=False,
    convolution_first=False,
    # Transformer Decoders
    transformer_decoder_emb=None,
    transformer_decoder_layers=0,
    transformer_decoder_heads=0,
    transformer_decoder_dim=0,
)

config = TrainingConfigV1(
    brain_encoder_config=model_config,
    data_partition=data_partition,
    # Ada lora settings
    # Around 100k total batches an epoch for gwilliams
    adalora_init_r=12,
    adalora_target_r=4,
    adalora_tinit=(400 * 3),  # 5% total steps
    adalora_tfinal=(400 * 8),  # 50-80% total steps
    adalora_deltaT=(400 * 1),  # 1-5% total steps
    adalora_lora_alpha=32,
    adalora_lora_dropout=0.1,
    adalora_total_step=(400 * 30),
    # Pre-processing parameters
    # Brain
    new_freq=200,
    frequency_bands={"all": (0.5, 80)},
    max_random_shift=1.0,
    window_size=4,
    window_stride=1,
    brain_clipping=None,
    baseline_window=0.5,
    notch_filter=True,
    scaling="both",
    delay=0.15,
    # Audio
    audio_model="openai/whisper-tiny.en",
    # Hyperparameters
    learning_rate=1e-4,
    weight_decay=1e-4,
    epochs=40,
    batch_size=128,
    random_test_size=10,
    seed=42,
    mel_alignment_objectives={
        "clip_loss": 0.6,
        "mse_loss": 0.4,
        "commitment_loss": 0.0,
        "cosine_similarity": 0.0,
    },
    latent_alignment_objectives={
        "cosine_similarity": 4.0,
        "mse_loss": 4.0,
        "clip_loss": 0.2,
    },
    latent_alignment_layers=[-1],
)

config.brain_encoder_config.mel_normalization = False
config.learning_rate = 1e-4
config.batch_size = 256

session = TrainingSessionV1(
    config=config,
    studies={study: "audio" for study in data_partition.keys()},
    data_path="/home/ubuntu/storage-texas/data",
    save_path="saves/phase2/architecture/task/transformers/4_transformer_layer",
    clear_cache=False,
    cache_name="cache",
    download_studies=False,
)

try:
    session.train(
        device="cuda",
        buffer_size=30,
        num_workers=(multiprocessing.cpu_count() - 2),
        max_cache_size=400,
        current_epoch=0,
    )
except KeyboardInterrupt as e:
    print("Exited")

# try:
#     session.pre_process_all_recordings(
#         buffer_size=30, num_workers=multiprocessing.cpu_count() - 2, max_cache_size=400
#     )
# except KeyboardInterrupt as e:
#     print("Exited")

Loading Gwilliams2023 with batch type audio
Data partitioned on studies ['gwilliams2023'].
Train: 135, Unseen Task: 12, Unseen Subject: 45, Unseen Both: 4.

RNNEncoder initialized as transformer with 4 layers, 384 d_model, 4 nhead
	Embedding: sinusoidal, params: 7097856
SimpleConv initialized with 16393840 parameters, cond: ['study', 'subject']
Merger False, merger channels 0
ConvBlocks: 6, hidden_dim: 384, params 8858112
Found 24 target modules for AdaLora: ['k_proj', 'q_proj', 'v_proj', 'out_proj', 'fc1', 'fc2']
openai/whisper-tiny.en loaded with 8540472 frozen params (4 layers and 384 dim).
AdaLora has 332064 trainable params.


Training Epoch 1: 100%|██████████| 135/135 [07:57<00:00,  3.54s/it]


Testing completed in 0.68m.
Epoch 1 completed in 8.64m. 0.06m per recording.
New highest average test accuracy: 0.0051, lowest FinLayer total loss: 4.9761 at epoch 1.


Training Epoch 2: 100%|██████████| 135/135 [07:56<00:00,  3.53s/it]


Testing completed in 0.67m.
Epoch 2 completed in 8.61m. 0.06m per recording.
New highest average test accuracy: 0.0049, lowest FinLayer total loss: 4.8695 at epoch 2.


...

Training Epoch 7: 100%|██████████| 135/135 [07:58<00:00,  3.54s/it]


Testing completed in 0.67m.
Epoch 7 completed in 8.64m. 0.06m per recording.
New highest average test accuracy: 0.0048, lowest FinLayer total loss: 4.5468 at epoch 7.


Training Epoch 8: 100%|██████████| 135/135 [07:57<00:00,  3.54s/it]


Testing completed in 0.66m.
Epoch 8 completed in 8.62m. 0.06m per recording.
New highest average test accuracy: 0.0048, lowest FinLayer total loss: 4.4408 at epoch 8.


Training Epoch 9:   0%|          | 0/135 [00:00<?, ?it/s]

In [ ]:
# for name, param in session.model.encoder.named_parameters():
#     if "lora_A" in name or "lora_B" in name or "lora_E" in name:
#         print(name)

In [ ]:
# import time

# dataloader = session.get_dataloader(buffer_size=30, num_workers=24, max_cache_size=400)
# dataloader.start_fetching(session.dataset["train"], cache=True)

# # Process batches as they become available
# try:
#     batches, recs, start_time = 0, 0, time.time()
#     print(f"Total recordings: {len(session.dataset['train'])}")

#     while True:
#         batch = dataloader.get_recording()

#         if batch is None:
#             break

#         brain = batch.brain_segments["all"]
#         batches += brain.shape[0]
#         recs += 1

#         if recs % 10 == 0:
#             print(f'Processed {recs} recordings of {len(session.dataset["train"])}')

#     print(
#         f"Batch {batches} ({recs} recordings) processed in {time.time() - start_time:.2f}s"
#     )
#     print(
#         f"Average processing time per recording: {(time.time() - start_time) / recs:.2f}s"
#     )
#     print(
#         f"Average processing time per batch: {(time.time() - start_time) / batches:.2f}s"
#     )

# except KeyboardInterrupt:
#     print("Interrupted")
#     dataloader.stop()
# except Exception as e:
#     print("Error", e)
#     dataloader.stop()